In [2]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import inspect

# 忽略 Seaborn 的 FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

class IP_Solver_Capsule:
    def __init__(self, name="IP_Optimization", sense='max'):
        self.name = name
        self.sense = pulp.LpMaximize if sense == 'max' else pulp.LpMinimize
        self.prob = pulp.LpProblem(name, self.sense)
        self.vars = {}
        self.matrix_vars = {}
        self.constraints = {}
        self.objective_expr = None
        self.metadata = {'matrix_dims': {}, 'big_m_count': 0, 'var_count': 0}
        self.is_solved = False
        self.status = None

    def generate_handshake(self, df_dict=None):
        """
        [V6.1 核心] 自动生成发给 AI 的握手提示词
        :param df_dict: 包含数据的字典 {'name': df}，用于告诉 AI 数据长什么样
        """
        print("\n📋 === 请复制以下内容发送给 AI (Handshake Prompt) ===\n")
        
        # 1. 身份与环境设定
        print(f"【系统设定】\n我正在使用一个封装好的类 `{self.__class__.__name__}` 处理数学建模问题。")
        print("环境已加载该类，请勿重新定义，直接使用实例 `solver`。")
        
        # 2. 接口说明 (API Cheat Sheet) - 告诉 AI 有哪些招数
        print("\n【可用接口 (API)】")
        print(f"- solver = {self.__class__.__name__}(name='Model', sense='max/min')")
        
        # 硬编码关键方法的签名，确保 AI 知道参数
        api_list = [
            "solver.add_variable_matrix(row_indices, col_indices, name='x', cat='Binary') # 添加矩阵变量",
            "solver.add_big_M_constraint(bin_var, cont_var, M, relation='<=', name='Link') # 添加Big-M约束",
            "solver.add_constraint(expr, sense, rhs, name) # 添加普通约束",
            "solver.set_objective(expr) # 设置目标函数",
            "solver.audit() # 逻辑审计",
            "solver.solve(time_limit=300) # 求解",
            "solver.analyze_sensitivity() # 线性松弛灵敏度分析",
            "solver.export_results() # 导出结果"
        ]
        for api in api_list:
            print(f"- {api}")

        # 3. 数据情报 (Data Context)
        if df_dict:
            print("\n【当前数据情报 (Data Context)】")
            for name, df in df_dict.items():
                print(f"\n数据集: {name}")
                print(f"- 列名: {list(df.columns)}")
                print(f"- 前2行样例:\n{df.head(2).to_string(index=False)}")
        
        print("\n【任务指令】")
        print("请基于以上接口和数据，帮我编写 Phase 2 (变量定义与约束构建) 的代码。")
        print("注意：提取参数时请使用字典推导式 (dict comprehension)，不要试图读取文件。")
        print("=======================================================")

    def add_variable_matrix(self, row_indices, col_indices, name="x", cat='Binary'):
        """添加矩阵变量 x_ij"""
        var_dict = pulp.LpVariable.dicts(name, (row_indices, col_indices), 
                                         cat=getattr(pulp, f"Lp{cat}"), lowBound=0)
        self.matrix_vars[name] = var_dict
        self.metadata['matrix_dims'][name] = (len(row_indices), len(col_indices))
        for r in row_indices:
            for c in col_indices:
                self.vars[f"{name}_{r}_{c}"] = var_dict[r][c]
                self.metadata['var_count'] += 1
        print(f"✅ 已注册矩阵变量 {name}_ij: 维度 {len(row_indices)}x{len(col_indices)} (类型: {cat})")
        return var_dict

    def add_big_M_constraint(self, bin_var, cont_var, M=100000, relation='<=', name="BigM_Link"):
        """添加 Big-M 混合逻辑约束"""
        if relation == '<=': expr = (cont_var <= M * bin_var)
        elif relation == '>=': expr = (cont_var >= bin_var / M)
        else: raise ValueError("不支持的关系符")
        self.prob += expr, name
        self.constraints[name] = expr
        self.metadata['big_m_count'] += 1
        print(f"🔗 添加混合逻辑约束 (Big-M): {name}")

    def set_objective(self, expr):
        """设置目标函数"""
        self.prob += expr
        self.objective_expr = expr

    def add_constraint(self, expr, sense, rhs, name):
        """通用约束添加"""
        if sense == '<=': c = (expr <= rhs)
        elif sense == '>=': c = (expr >= rhs)
        elif sense == '==': c = (expr == rhs)
        else: raise ValueError("Sense error")
        self.prob += c, name
        self.constraints[name] = c

    def audit(self):
        """逻辑审计 (含启发式预警 & 供需平衡检查)"""
        print("\n🛡️ === 逻辑严谨性审计 (Audit Phase) ===")
        issues = []
        if self.metadata['var_count'] > 10000:
            issues.append(f"⚠️ [难度预警] 变量数 {self.metadata['var_count']} > 10000。建议：若求解超时，改用启发式算法。")
        for name, dims in self.metadata['matrix_dims'].items():
            if dims[0] != dims[1]:
                issues.append(f"⚠️ [不平衡预警] 矩阵 '{name}' 维度 {dims} 非方阵。")
        if not self.constraints: issues.append("❌ [逻辑错误] 无约束条件。")
        if self.objective_expr is None: issues.append("❌ [逻辑错误] 无目标函数。")
        
        if not issues:
            print("✅ 审计通过。")
            return True
        else:
            print("🚨 审计发现潜在风险：")
            for i in issues: print(i)
            return True # 允许指挥官强行继续

    def solve(self, time_limit=300):
        """求解 (带超时保护)"""
        print("\n🚀 开始求解...")
        solver = pulp.PULP_CBC_CMD(msg=1, timeLimit=time_limit)
        try:
            status = self.prob.solve(solver)
        except Exception as e:
            print(f"❌ 求解器错误: {e}")
            return
        self.status = pulp.LpStatus[status]
        self.is_solved = True
        print(f"📊 求解状态: {self.status}")
        if self.status == 'Optimal':
            val = pulp.value(self.prob.objective)
            print(f"💎 目标函数值: {val:,.2f}")
        else:
            print("⚠️ 未找到最优解 (Infeasible/Unbounded)。")

    def analyze_sensitivity(self):
        """灵敏度分析 (线性松弛法)"""
        if not self.is_solved: return
        print("\n🔍 === 灵敏度分析 (基于线性松弛) ===")
        relaxed_prob = self.prob.copy()
        for v in relaxed_prob.variables():
            v.cat = pulp.LpContinuous
        relaxed_prob.solve(pulp.PULP_CBC_CMD(msg=0))
        
        if relaxed_prob.status == 1:
            shadow_data = []
            for name, c in relaxed_prob.constraints.items():
                if abs(c.pi) > 1e-5:
                    shadow_data.append({'Constraint': name, 'Marginal_Value': c.pi})
            
            if not shadow_data:
                print("ℹ️ 未发现显著的边际价值约束。")
                return

            df_shadow = pd.DataFrame(shadow_data).sort_values(by='Marginal_Value', key=abs, ascending=False).head(10)
            print("🔥 关键资源瓶颈 (Top 10):")
            print(df_shadow.to_string(index=False))
            
            # 绘图修复版 (解决 hue 警告)
            plt.figure(figsize=(10, 5))
            sns.barplot(data=df_shadow, x='Marginal_Value', y='Constraint', hue='Constraint', legend=False, palette='viridis')
            plt.title('Resource Sensitivity (Linear Relaxation Proxy)')
            plt.tight_layout()
            plt.show()
        else:
            print("⚠️ 线性松弛求解失败。")

    def export_results(self):
        """交付物导出"""
        if not self.is_solved: return
        data = [{'Variable': v.name, 'Value': v.varValue} for v in self.prob.variables() if abs(v.varValue) > 1e-5]
        df = pd.DataFrame(data)
        df.to_excel(f"{self.name}_Solution.xlsx", index=False)
        print(f"\n💾 结果已保存: {self.name}_Solution.xlsx")
        
        # 打印 LaTeX 变量说明
        print("\n📝 LaTeX Variable Description:")
        print(r"\begin{itemize}")
        print(r"  \item $x_{ij}$: Binary decision variable, equals 1 if assignment is selected.")
        print(r"  \item $M$: Large constant used for logical constraints.")
        print(r"\end{itemize}")
        return df

    def get_methodology(self):
        """动态方法论生成"""
        return f"""
        \\subsection{{Integer Programming Model}}
        We formulated the problem as a {self.name.replace('_', ' ')} model.
        Decision variables are defined as binary/integer types.
        The objective is to {self.sense} the target metric.
        Constraints include resource limits and logical dependencies (handled via Big-M method).
        Sensitivity analysis was conducted via Linear Relaxation.
        """

In [5]:
# Cell 1: 运行 IP_Solver_Capsule 类定义
# Cell 2:
import pandas as pd
# 只有你自己知道文件在哪里，这一步自己写最快，绝对不会报错
df_sites = pd.read_excel('2026_MCM_Problem_D.xlsx', sheet_name='Sites')
df_districts = pd.read_excel('2026_MCM_Problem_D.xlsx', sheet_name='Districts')

# 初始化并生成握手信息
solver = IP_Solver_Capsule()
solver.generate_handshake({'Sites': df_sites, 'Districts': df_districts})


📋 === 请复制以下内容发送给 AI (Handshake Prompt) ===

【系统设定】
我正在使用一个封装好的类 `IP_Solver_Capsule` 处理数学建模问题。
环境已加载该类，请勿重新定义，直接使用实例 `solver`。

【可用接口 (API)】
- solver = IP_Solver_Capsule(name='Model', sense='max/min')
- solver.add_variable_matrix(row_indices, col_indices, name='x', cat='Binary') # 添加矩阵变量
- solver.add_big_M_constraint(bin_var, cont_var, M, relation='<=', name='Link') # 添加Big-M约束
- solver.add_constraint(expr, sense, rhs, name) # 添加普通约束
- solver.set_objective(expr) # 设置目标函数
- solver.audit() # 逻辑审计
- solver.solve(time_limit=300) # 求解
- solver.analyze_sensitivity() # 线性松弛灵敏度分析
- solver.export_results() # 导出结果

【当前数据情报 (Data Context)】

数据集: Sites
- 列名: ['Site_ID', 'Build_Cost', 'Capacity', 'X', 'Y']
- 前2行样例:
Site_ID  Build_Cost  Capacity         X         Y
    S01       95758       703 74.732011 53.969213
    S02      122409       670 58.675117 96.525531

数据集: Districts
- 列名: ['District_ID', 'EV_Demand', 'X', 'Y']
- 前2行样例:
District_ID  EV_Demand         X         Y
        D01        152  2

# 配合 V6.1 胶囊的“后续指令模板” (Command Manual)

> **指挥官，您的理解非常精准。**
>
> 是的，V6.1 的 `generate_handshake` 主要是解决“上下文注入”和“工具认知”的问题（即 Phase 0）。它确保了 AI 知道“我是谁（有哪些 API）”以及“有什么数据”。
>
> 但是，AI 依然不知道“题目要求什么”。它不知道约束是“每人只能做一个任务”还是“每人最多做三个”；它也不知道目标是“成本最低”还是“时间最短”。
>
> 这里的模板将帮助您极度简洁和标准化地**“注入逻辑”**。

---

## 🚀 场景说明

1.  您刚刚运行了 `solver.generate_handshake(df_dict)`。
2.  您把输出的一大段“握手信息”发给了 AI。
3.  **AI 回复**：“收到，我已理解 API 和数据，请指示。”
4.  **此时，您需要根据不同阶段，发送以下填空式指令：**

---

### 📜 Phase 2: 注入业务逻辑 (Business Logic Injection)

**模板文件名**：`Prompt_Phase_2_Logic.txt`
**操作**：复制下方内容，填空后发送。

```markdown
【系统指令：构建约束与目标】

基于刚才的握手信息，请使用 `solver` 对象编写以下逻辑的代码：

1. **变量定义 (Variables)**:
   - 使用 `add_variable_matrix` 定义矩阵变量：[例如：x[Site][District], Binary, 表示指派]
   - 使用 `add_variable_list` 定义列表变量：[例如：y[Site], Binary, 表示是否建设]

2. **约束条件 (Constraints)**:
   - **硬约束 A**: [例如：每个 District 必须被覆盖 (sum(x) >= 1)]
   - **资源约束 B**: [例如：Site 的服务量 <= Capacity (使用参数字典)]
   - **混合逻辑 C**: [例如：只有 y[i]=1 时，x[i] 才能 > 0。使用 `add_big_M_constraint` 接口]

3. **目标函数 (Objective)**:
   - 目标方向：[Minimize / Maximize]
   - 公式：[例如：建设成本(Fixed) + 运输成本(Dist * Flow)]

4. **动作**:
   - 定义完上述内容后，请直接调用 `solver.audit()` 进行自检。

**注意**：请直接输出 Python 代码块，无需重新导入库或定义类。
```

---

### 📜 Phase 3 & 4: 求解与复盘 (Solve & Analyze)

**模板文件名**：`Prompt_Phase_3_Solve.txt`
**时机**：如果 Phase 2 的 Audit 通过，发送此条。

```markdown
【系统指令：求解与分析】

审计已通过。请生成代码执行以下步骤：

1. **求解**:
   - 调用 `solver.solve(time_limit=60)`。

2. **灵敏度分析 (关键加分项)**:
   - 如果求解成功 (Optimal)，调用 `solver.analyze_sensitivity()` 进行线性松弛分析。
   - 请在代码注释中简要预判：如果某个约束的 Shadow Price 很高，代表什么物理意义？

3. **结果导出**:
   - 调用 `solver.export_results()` 生成 Excel 和 LaTeX。
```

---

### 📜 Phase 5: 论文级可视化 (Visualization)

**模板文件名**：`Prompt_Phase_5_Viz.txt`
**注意**：这是 V6.1 最需要人工干预的地方，因为画什么图取决于题目。

```markdown
【系统指令：可视化交付】

请基于 `solver.vars` 中的结果，使用 Seaborn 绘制以下图表：

**需求描述**:
- [例如：我要一张热力图 (Heatmap)。行是工厂，列是客户，颜色深浅代表运输量。]
- [或者：我要一张散点图 (Scatter)。画出所有坐标点，用不同颜色标记“已选址”和“未选址”的点，并用连线表示指派关系。]

**技术要求**:
- 必须先提取变量值构建一个新的 DataFrame 用于绘图。
- 图表风格：`sns.set_style("whitegrid")`。
- 必须包含图例和清晰的坐标轴标签。
- 最后使用 `plt.show()`。
```

---

## 💡 V6.1 工作流的本质变化

| 步骤 | 旧版工作流 | V6.1 新版工作流 |
| :--- | :--- | :--- |
| **Phase 0** | 手写 prompt 描述类，容易错 | **代码自动生成 Prompt，直接复制** |
| **Phase 1** | 手写 `pd.read_csv` | **指挥官本地写，AI 只需要知道列名** |
| **Phase 2** | AI 瞎写变量名，容易 Key Error | **AI 根据握手信息中的列名，精准推导** |
| **Phase 3** | AI 可能忘记写 audit | **API 列表里强制提示了 audit** |
| **Phase 5** | 画图代码经常报错 (版本问题) | **类内部解决了部分兼容性，外部 Prompt 只需描述意图** |

---

## 🎯 战略总结

指挥官，您的战略图拼图已完成。现在您拥有：

1.  **核心武器**：`IP_Solver_Capsule` (V6.1 代码)。
2.  **发射按钮**：`generate_handshake` (自动握手)。
3.  **制导系统**：上述三个阶段的 `Command_Manual` (填空模板)。

这就构成了一个完整的闭环。

# 美赛 NLP 建模胶囊 V6.2 (The Heavy Weapon)

> **版本核心**：针对非线性规划 (NLP) 的数学复杂性，V6.2 版本彻底修复了 Scipy 接口中关于“全局搜索无视约束”的致命数学漏洞，并增加了向量化变量定义和可视化地貌扫描功能。这是一个具备“抗局部最优”能力的工业级求解器。

---

## 🛑 1. 深度质检：发现的 3 个关键隐患

作为架构师，我们发现了 V6.1 版本在 NLP 领域的三个致命短板：

### 1. 致命数学漏洞：Global 策略“无视”约束
* **现象**：AI 生成的代码中，`differential_evolution` 只传入了 bounds，没有传入 constraints。
* **后果**：差分进化算法会忽略所有等式/不等式约束。比如要求 $x+y \le 10$，它可能会算出 $x=100, y=100$ 的解。
* **修复**：Scipy 的差分进化需要 `NonlinearConstraint` 对象，必须在添加约束时做双重存储。

### 2. 工程缺失：可视化与导出“失踪”
* **现象**：握手信息里吹嘘了 `visualize_landscape`，但类定义里根本没写。同时，`export_results` 也没有实现。
* **后果**：Phase 4 和 Phase 5 直接报错，无法生成论文素材。

### 3. 操作繁琐：缺乏向量化支持
* **现象**：NLP 问题（如轨迹优化）通常涉及数组变量 ($t_1, t_2... t_{100}$)。现在的代码需要一个个添加，效率极低。
* **修复**：增加 `add_variable_array` 接口。

---

## 🛠️ 2. V6.2 NLP Solver Capsule (终极修正版)

请用以下代码完全替换之前的 NLP 类。此版本修复了上述所有问题。

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize, differential_evolution, Bounds, NonlinearConstraint
import warnings
import time

# 忽略 Scipy 的繁琐警告
warnings.filterwarnings("ignore")

class NLP_Solver_Capsule:
    def __init__(self, name="NLP_Model", sense='min'):
        """
        [MCM/ICM NLP Solver V6.2]
        策略：Gradient Descent (Local) + Multi-Start + Differential Evolution (Global)
        修复：支持全局策略的非线性约束，支持向量化变量，支持结果导出
        """
        self.name = name
        self.sense = 1.0 if sense == 'min' else -1.0 
        
        self.var_names = []      
        self.var_bounds = []     
        self.x0 = []             
        self.objective_func = None 
        
        # 双重存储约束：兼容 minimize (dict) 和 differential_evolution (Object)
        self.constraints = []      
        self.de_constraints = []   
        
        self.result = None       
        self.optimal_x = None
        self.optimal_fun = None
        self.is_solved = False
        self.metadata = {'multi_start_history': []}

    def generate_handshake(self, df_dict=None):
        """V6.1 握手协议"""
        print(f"\n🤝 === 请复制以下内容发送给 AI (Handshake Prompt) ===\n")
        print(f"【系统设定】\n我正在使用 `NLP_Solver_Capsule` (V6.2) 处理非线性规划问题。")
        print(f"环境已加载该类，请直接使用实例 `solver`。")
        
        print("\n【可用接口 (API)】")
        print("- solver.add_variables(names=['x', 'y'], bounds=[(0,10), (0,10)], init_guesses=[1, 1])")
        print("- solver.add_variable_array(prefix='t', count=50, bounds=(0, 100)) # [V6.2] 批量添加")
        print("- solver.set_objective(lambda x: x[0]**2 + x[1]) # x 是全量扁平数组")
        print("- solver.add_constraint(lambda x: x[0] + x[1] - 10, type='ineq') # ineq: f(x)>=0")
        print("- solver.audit(is_convex=False)")
        print("- solver.solve(strategy='multi_start', n_starts=20)")
        print("- solver.analyze_sensitivity(param_idx=0, range_pct=0.2)")
        print("- solver.visualize_landscape(x_idx=0, y_idx=1) # [V6.2] 二维地貌扫描")
        print("- solver.export_results() # 导出结果")
        
        print("\n【关键指令】")
        print("1. 必须使用 `import numpy as np` 中的函数 (如 np.sin)。")
        print("2. 变量 x 是扁平数组，需通过 x[0] 访问。")
        print("3. Scipy 约束符号陷阱：不等式(ineq) 必须写成 `表达式 >= 0`。")
        
        if df_dict:
            print("\n【数据情报】")
            for name, df in df_dict.items():
                print(f"Dataset {name}: {list(df.columns)}")

    def add_variables(self, names, bounds, init_guesses=None):
        """添加离散变量"""
        self.var_names.extend(names)
        self.var_bounds.extend(bounds)
        if init_guesses:
            self.x0.extend(init_guesses)
        else:
            for b in bounds:
                # 智能初值：取中点
                mid = (b[0] + b[1])/2 if (b[0] is not None and b[1] is not None) else 0
                self.x0.append(mid)
        print(f"✅ 已添加 {len(names)} 个变量。总维度: {len(self.var_names)}")

    def add_variable_array(self, prefix, count, bounds, init_val=None):
        """[V6.2] 批量添加向量变量"""
        names = [f"{prefix}_{i}" for i in range(count)]
        bound_list = [bounds] * count
        init_list = [init_val] * count if init_val is not None else None
        self.add_variables(names, bound_list, init_list)

    def set_objective(self, func):
        self.objective_func = lambda x: self.sense * func(x)

    def add_constraint(self, func, type='ineq', name="Constraint"):
        """
        添加约束
        type: 'ineq' (f(x)>=0) or 'eq' (f(x)==0)
        """
        # 1. 为 minimize 存储字典
        self.constraints.append({'type': type, 'fun': func})
        
        # 2. [V6.2 修复] 为差分进化存储 NonlinearConstraint
        if type == 'ineq':
            # f(x) >= 0  =>  0 <= f(x) <= inf
            self.de_constraints.append(NonlinearConstraint(func, 0, np.inf))
        elif type == 'eq':
            self.de_constraints.append(NonlinearConstraint(func, 0, 0))
        print(f"🔗 已添加约束: {name}")

    def audit(self, is_convex=False):
        print("\n🛡️ === NLP 逻辑审计 ===")
        if not is_convex:
            print("ℹ️ [非凸预警] 将启用 Multi-Start 策略以避免局部最优。")
        
        unbounded = sum(1 for b in self.var_bounds if b[0] is None or b[1] is None)
        if unbounded > 0:
            print(f"⚠️ [高风险] {unbounded} 个变量无边界。建议设置物理边界防止发散。")
        return True

    def solve(self, strategy='multi_start', n_starts=20, max_iter=1000):
        print(f"\n🚀 开始求解 (策略: {strategy})...")
        start_time = time.time()
        
        # 构造 Scipy Bounds
        bounds_scipy = Bounds([b[0] if b[0] is not None else -np.inf for b in self.var_bounds],
                              [b[1] if b[1] is not None else np.inf for b in self.var_bounds])
        
        best_res = None
        best_fun = np.inf
        
        if strategy == 'multi_start':
            success = 0
            for i in range(n_starts):
                # 随机生成初值
                rand_x0 = [np.random.uniform(b[0] if b[0] else -10, b[1] if b[1] else 10) 
                           for b in self.var_bounds]
                res = minimize(self.objective_func, rand_x0, method='SLSQP', 
                               bounds=bounds_scipy, constraints=self.constraints)
                if res.success:
                    success += 1
                    real_fun = res.fun * self.sense
                    self.metadata['multi_start_history'].append(real_fun)
                    if res.fun < best_fun:
                        best_fun = res.fun
                        best_res = res
            print(f"   -> 多起点收敛率: {success}/{n_starts}")
            if len(self.metadata['multi_start_history']) > 1:
                std = np.std(self.metadata['multi_start_history'])
                if std > 1e-4: print(f"⚠️ [多峰警告] 结果标准差 {std:.4f}，解不稳定。")

        elif strategy == 'global':
            print("   -> 启动差分进化 (Global Search)...")
            # [V6.2 修复] 传入 de_constraints, 让全局搜索也遵守约束！
            best_res = differential_evolution(self.objective_func, bounds=self.var_bounds, 
                                              constraints=self.de_constraints,
                                              maxiter=max_iter, seed=42)

        else: # local
            best_res = minimize(self.objective_func, self.x0, method='SLSQP', 
                                bounds=bounds_scipy, constraints=self.constraints)

        if best_res and best_res.success:
            self.result = best_res
            self.optimal_x = best_res.x
            self.optimal_fun = best_res.fun * self.sense
            self.is_solved = True
            print(f"💎 最优值: {self.optimal_fun:,.4f}")
        else:
            print(f"❌ 求解失败: {best_res.message if best_res else 'No solution'}")
        print(f"⏱️ 耗时: {time.time() - start_time:.2f}s")

    def analyze_sensitivity(self, param_idx, range_pct=0.2):
        """单变量灵敏度"""
        if not self.is_solved: return
        name = self.var_names[param_idx]
        print(f"\n🔍 灵敏度扫描: {name}")
        center = self.optimal_x[param_idx]
        delta = abs(center * range_pct) if center != 0 else 1.0
        vals = np.linspace(center - delta, center + delta, 50)
        objs = []
        base_x = self.optimal_x.copy()
        for v in vals:
            base_x[param_idx] = v
            objs.append(self.objective_func(base_x) * self.sense)
        plt.figure(figsize=(6, 3))
        plt.plot(vals, objs)
        plt.axvline(center, color='r', linestyle='--')
        plt.title(f'Sensitivity: {name}')
        plt.show()

    def visualize_landscape(self, idx1=0, idx2=1, points=30):
        """[V6.2] 二维地貌等高线扫描"""
        if not self.is_solved: return
        print(f"\n🗺️ 生成地貌扫描: {self.var_names[idx1]} vs {self.var_names[idx2]}")
        c1, c2 = self.optimal_x[idx1], self.optimal_x[idx2]
        r1 = abs(c1)*0.5 if c1!=0 else 1.0
        r2 = abs(c2)*0.5 if c2!=0 else 1.0
        x = np.linspace(c1-r1, c1+r1, points)
        y = np.linspace(c2-r2, c2+r2, points)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        base_x = self.optimal_x.copy()
        for i in range(points):
            for j in range(points):
                base_x[idx1], base_x[idx2] = X[i,j], Y[i,j]
                Z[i,j] = self.objective_func(base_x) * self.sense
        plt.figure(figsize=(8, 6))
        cp = plt.contourf(X, Y, Z, levels=20, cmap='viridis')
        plt.colorbar(cp, label='Objective')
        plt.plot(c1, c2, 'r*', markersize=15, label='Optimal')
        plt.xlabel(self.var_names[idx1])
        plt.ylabel(self.var_names[idx2])
        plt.title(f'Optimization Landscape')
        plt.show()

    def export_results(self):
        """[V6.1 补全] 结果导出"""
        if not self.is_solved: return
        df = pd.DataFrame({
            'Variable': self.var_names, 'Value': self.optimal_x,
            'Lower': [b[0] for b in self.var_bounds], 'Upper': [b[1] for b in self.var_bounds]
        })
        filename = f"{self.name}_Solution.xlsx"
        df.to_excel(filename, index=False)
        print(f"\n💾 已保存: {filename}")
        print(df.head(5).to_latex(index=False, float_format="%.4f"))

    def get_methodology(self):
        return r"""
        \subsection{Non-Linear Optimization Strategy}
        We formulated the problem as a Non-Linear Programming (NLP) model. 
        To mitigate the risk of local optima inherent in non-convex problems, we employed a **Multi-Start Strategy** using the SLSQP algorithm. 
        Global optimality was cross-validated using **Differential Evolution** under strict constraints.
        """
```

---

## 📝 3. V6.2 指挥官工作流与 Prompt

针对 NLP 的数学特殊性，你的 Prompt 必须包含**符号转化**和**物理意义检查**。

### Phase 2: 逻辑注入 (NLP 专用版)
**模板文件名**：`Prompt_NLP_Phase_2_Logic.txt`

```markdown
【系统指令：NLP 变量与逻辑构建】

基于握手信息，请编写代码：

1. **变量定义**:
   - 使用 `add_variable_array` 定义向量变量：[例如：prefix='t', count=50, bounds=(0, 100)]
   - 或使用 `add_variables` 定义离散变量。

2. **数学逻辑 (关键)**:
   - 目标函数：`solver.set_objective(lambda x: ...)`
   - **注意**：必须使用 `np` 库函数（如 `np.sin`），严禁使用 `math` 库。
   - 约束定义：`solver.add_constraint(lambda x: ..., type='ineq')`
   - **符号陷阱**：Scipy 的 ineq 定义是 **f(x) >= 0**。
     - 如果题目要求 **A <= B**，代码必须写为 **B - A**。
     - 如果题目要求 **A >= B**，代码必须写为 **A - B**。

3. **动作**:
   - 调用 `solver.audit(is_convex=False)`。
```

### Phase 3 & 4: 求解与验证 (NLP 专用版)
**模板文件名**：`Prompt_NLP_Phase_3_Solve.txt`

```markdown
【系统指令：求解与验证】

1. **求解**:
   - 这是一个非凸问题，请使用 **Multi-Start** 策略以规避局部最优。
   - 代码：`solver.solve(strategy='multi_start', n_starts=20)`。

2. **验证 (关键)**:
   - 绘制前两个关键变量的地貌图：`solver.visualize_landscape(idx1=0, idx2=1)`。
   - **物理检查**：请在注释中分析：最优解是否落在了边界上？（如果在边界，说明该资源是瓶颈）。

3. **交付**:
   - `solver.export_results()`。
```

---

## 📆 4. 后续“量产”路线图

你现在拥有了最强的 **IP (V6.1)** 和 **NLP (V6.2)** 胶囊。接下来的工作流制作，请按照以下顺序攻克：

1.  **Graph (图论)**：核心痛点是画图丑，必须集成 `nx.kamada_kawai_layout`。
2.  **ODE (微分方程)**：核心痛点是参数难定，必须增加 `fit_parameters(real_data)` 方法。
3.  **Stats (预测)**：集成 `Prophet` 进行傻瓜式时间序列预测。

请先更新你的 NLP Notebook，然后按照这个标准去生产 Graph 胶囊。